In [0]:
#Ötlet: x_list.grad csak egy skalár vektort ad, requires_grad_=False, nem tudja rendesen figyelembe venni a tanuláshoz
#Helyette: közelítő derivált?
#Der(1, f, del, x) = (f(x+del/2)-f(x-del/2))/del
#Der(2, f, del, x) = (f(x)-f(x-del))/del
#Der(3, f, del, x) = (f(x+del)-f(x))/del

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch as th
import torch.nn as nn
import torch.autograd
import time

In [0]:
a = th.tensor([1.0], requires_grad=True)
b = th.tensor([2.0], requires_grad=True)
x = th.tensor([3.0], requires_grad=True)
y = a*x*x + b
d = torch.autograd.grad([y], [x], only_inputs=True, create_graph=True)[0]
w = d + y
w.backward()
print(a, b, x, y, d, w,"\n")
print(a.grad, b.grad, x.grad, y.grad, d.grad, w.grad,"\n")

#magasabb foku derivaét?

mulg = torch.autograd.grad([y],[a,b,x],only_inputs=True,create_graph=True, retain_graph=True)
print("mulg:", mulg)
#mulg[1] = 1, hibát ad ha gradot számolunk
#print("mulg autograd: ", torch.autograd.grad(mulg,[a,b,x],only_inputs=True,create_graph=True, allow_unused=True))
print("mulg[0] autograd:",torch.autograd.grad([mulg[0]],[a,b,x],only_inputs=True,create_graph=True, allow_unused=True))
print("mulg[2] autograd", torch.autograd.grad([mulg[2]],[a,b,x],only_inputs=True,create_graph=True, allow_unused=True))
print("[mulg[0],mulg[2]] autograd: ", torch.autograd.grad([mulg[0], mulg[2]],[a,b,x],only_inputs=True,create_graph=True, allow_unused=True))
print("összeadja")

tensor([1.], requires_grad=True) tensor([2.], requires_grad=True) tensor([3.], requires_grad=True) tensor([11.], grad_fn=<AddBackward0>) tensor([6.], grad_fn=<AddBackward0>) tensor([17.], grad_fn=<AddBackward0>) 

tensor([15.]) tensor([1.]) tensor([8.]) None None None 



RuntimeError: ignored

In [0]:
xdim = th.tensor([1.0, 2.0, 3.01], requires_grad=True)
ydim = xdim[0]*xdim[1]*xdim[2] + th.sum(xdim[0:1] ** xdim[0:1])
th.autograd.grad(ydim, xdim, only_inputs=True, create_graph=True, retain_graph=True)

(tensor([7.0200, 3.0100, 2.0000], grad_fn=<AddBackward0>),)

In [0]:
xd = th.tensor([1.0, 2.0], requires_grad=True)
yd = th.sum(xd ** xd)

In [0]:
#többdim?
A = th.tensor([1.0, 2.0],requires_grad=True)
B = A + th.tensor([0.0, 1.]) * A * A
s = A.size()
D = th.autograd.grad(B, A, grad_outputs=th.ones(A.size()), only_inputs=True, create_graph=True)[0]
print(D)

tensor([1., 5.], grad_fn=<AddBackward0>)


In [0]:
model = nn.Sequential(
    nn.Linear(1, 100),
    nn.Linear(100, 1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,100),
    nn.ReLU(),
    nn.Linear(100,100),
    nn.Linear(100,1)
)

In [0]:
#itt lehet definiálni, milyen differenciál egyenlet megoldását próbáljuk közelíteni, F(t,x,...) = 0 formában a bal oldal kell
def ODE(x):
  u = model(x)
  deriv = th.autograd.grad(u, x, grad_outputs=th.ones(x.size()), only_inputs=True, create_graph=True)[0]  #u'(x)
  leftside = deriv - u * th.sin(x) # u'(x)-u(x)sin(x)
  return leftside


In [0]:
def solution(x):
  r = th.cos(x)
  r = th.exp(-r)
  return r

In [0]:
def optim(x, lr = 1e-3):
  opt = th.optim.Adam(model.parameters(), lr=lr)

  opt.zero_grad()
  
  u = model(x)
  deriv = th.autograd.grad(u, x, grad_outputs=th.ones(x.size()), only_inputs=True, create_graph=True)[0]
  h = deriv - u * th.sin(x)
  h = sum(h*h)
  
  opt.zero_grad()
  h.backward()
  opt.step()


In [0]:
point_count = 10000
tan_pont = th.arange(start = 0, end = point_count)
tan_pont = tan_pont / (point_count * 1.0)
tan_pont.requires_grad = True
tan_pont = tan_pont.reshape((point_count,1))
testcas = th.sin(2* tan_pont)-2
testcas.backward(th.ones(tan_pont.size()))
print(tan_pont.requires_grad)
print(testcas)

True
tensor([[-2.0000],
        [-1.9998],
        [-1.9996],
        ...,
        [-1.0905],
        [-1.0905],
        [-1.0906]], grad_fn=<SubBackward0>)


In [0]:
x_example = th.tensor([2.])
x_example.requires_grad = True
h = x_example**2-th.sin(x_example)
h.backward()
x_example.grad

tensor([4.4161])

In [0]:
#lehet ha először numerikus deriváltal számolunk, gyorsabban közel jut
tan_pont.requires_grad_(False)
startnum = time.time()
for i in range(100):
  numoptim(tan_pont, 1e-5 , 1e-6)
endnum = time.time()
print(endnum - startnum, "sec")
print(time.gmtime(endnum - startnum))
print((endnum - startnum)/100, "sec on average")

RuntimeError: ignored

In [0]:
tan_pont.requires_grad_(True)
opt_count = 10
start = time.time()
for i in range(opt_count):
  optim(tan_pont, 1e-7)
end = time.time()
print(end - start, "sec")
print(time.gmtime(end - start))
print((end - start)/opt_count, "sec on average")

In [0]:
plotpnt = tan_pont.reshape(tan_pont.size(0))
plt.plot(plotpnt.detach().numpy(), solution(plotpnt).detach().numpy())
scale = solution(tan_pont[0])/model(tan_pont[0])
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), (scale * model(tan_pont).reshape(tan_pont.size(0))).detach().numpy())

In [0]:
def numder(func, x, eps):
  x_1 = x - eps
  x_2 = x
  r = func(x_2)-func(x_1)
  r = r / eps
  return r

In [0]:
def numoptim(x_list, eps, lr = 1e-3):
  opt = th.optim.Adam(model.parameters(), lr=lr) #esetleg más algoritmus?

  opt.zero_grad()
  
  x_grad = numder(model, x_list, eps)
  h = x_grad - model(x_list)*th.sin(x_list)
  h = sum(h*h)
  
  h.backward()
  opt.step()

In [0]:
def der(func, x):
  x.requires_grad(True)
  val = func(x)
  d = th.autograd.grad([val], [x],only_inputs=True, create_graph=True)[0]
  return d

In [0]:
#du/dx - u*sin(x) = 0
#ötlet: Adott D(u(x)) = 0, ahol D valami diff-op+függvény pl D(u(x)) = u''(x)-u(x)/(u'(x)-x^2)+sin(e^x)
#Ha u: R->R, akkor adott N: R->R neuronháló jó közelítés lehet, ha elég sűrű helyeken teljesíti D(N(x))=0 -->tanítás Sum(x_i, D(N(x_i))^2) hibáfüggvénnyel?

#konkrét példa:
#ODE: (du/dx)(x) - u(x)*sin(x) = 0 ,tfh x in [0,1], u(0)=1

In [0]:
#A neuronháló definiálása
model1 = nn.Sequential(
    nn.Linear(1, 100),
    nn.Linear(100, 1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,100),
    nn.ReLU(),
    nn.Linear(100,100),
    nn.Linear(100,1)
)

model2 = nn.Sequential(
    nn.Linear(1, 100),
    nn.Linear(100, 1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,100),
    nn.ReLU(),
    nn.Linear(100,100),
    nn.Linear(100,1)
)

model3 = nn.Sequential(
    nn.Linear(1, 100),
    nn.Linear(100, 1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,1000),
    nn.ReLU(),
    nn.Linear(1000,1000),
    nn.Linear(1000,100),
    nn.ReLU(),
    nn.Linear(100,100),
    nn.Linear(100,1)
)

In [0]:
#Ha lehet, jó lenne az ODE-t úgy definiálnihogy hsználható legyen a hibafüggvényben
#def ODE(func, x):
#  result = 
#  return result
#lehet csak közelítve tudunk majd számolni
#def ODE(der_type, func, x, eps):  
#szép deriváltak->talán sympy-vel?
  

In [0]:
# optimalizációs lépés, kézi derivalt kozelites, ebből tudnia kell paraméter-grad-ot számolni
# der-type: 1->+/- eps/2, stb
def optim(der_type, NN, x_list, DerivEps = 1e-3, lr = 1e-2):
  
  opt = th.optim.Adam(NN.parameters(), lr=lr) #esetleg más algoritmus?

  opt.zero_grad()
  
  x_grad = deriv(der_type, NN, x_list, DerivEps)
  h = x_grad - NN(x_list)*th.sin(x_list)
  h = sum(h*h)
  
  h.backward()
  opt.step()

In [0]:
#tanítási pontok, lehetne véletlenszeresíteni is
tan_pont = th.tensor(range(1001))*0.001
tan_pont = tan_pont.reshape(1001,1)
print(tan_pont.size())
tan_pont.requires_grad_()
tan_pont
tan_pont.backward(th.ones(tan_pont.size()))

In [0]:
#tanítás
learn_steps = 50
start = time.time()
for i in range(learn_steps):
  optim(1, model1, tan_pont, 1e-4, 1e-7)
  optim(2, model2, tan_pont, 1e-4, 1e-7)
  optim(3, model3, tan_pont, 1e-4, 1e-7)
end = time.time()
print(end-start, "sec")

In [0]:
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), model1(tan_pont).reshape(tan_pont.size(0)).detach().numpy())
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), model2(tan_pont).reshape(tan_pont.size(0)).detach().numpy())
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), model3(tan_pont).reshape(tan_pont.size(0)).detach().numpy())


In [0]:
scale1 = th.tensor(solution(plotpnt[0]))/model1(tan_pont[0])
print(scale1)
scale1 = scale1.detach().numpy()
plt.plot(plotpnt, solution(plotpnt))
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), scale1*model1(tan_pont).reshape(tan_pont.size(0)).detach().numpy())

In [0]:
scale2 = th.tensor(solution(plotpnt[0]))/model2(tan_pont[0])
print(scale2)
scale2 = scale2.detach().numpy()
plt.plot(plotpnt, solution(plotpnt))
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), scale2*model2(tan_pont).reshape(tan_pont.size(0)).detach().numpy())

In [0]:
scale3 = th.tensor(solution(plotpnt[0]))/model3(tan_pont[0])
print(scale3)
scale3 = scale3.detach().numpy()
plt.plot(plotpnt, solution(plotpnt))
plt.plot(tan_pont.reshape(tan_pont.size(0)).detach().numpy(), scale3*model3(tan_pont).reshape(tan_pont.size(0)).detach().numpy())

In [0]:
#elavult, .grad-os módszer
#optimalizációs lépés
def optim1(x_list, NN, lr = 1e-2):
  
  opt = th.optim.Adam(NN.parameters(), lr=lr) #esetleg más algoritmus?
  opt.zero_grad()
  x_list.grad.data.zero_()
  NN(x_list).backward(th.ones(x_list.size()))
  x_grad = x_list.grad
  h = x_grad - NN(x_list)*th.sin(x_list)
  h = sum(h*h)
  
  opt.zero_grad() # Az x-grad számolás lehet elrontja a súlyok grad-ját
  
  h.backward()
  opt.step()
  

In [0]:
#különböző deriváltak számolásához
def deriv(der_type, func, x, eps):
  if der_type == 1:
    x_1 = x - eps/2
    x_2 = x + eps/2
    r = func(x_2)-func(x_1)
    r = r / eps
    return r
  elif der_type == 2:
    x_1 = x - eps
    x_2 = x
    r = func(x_2)-func(x_1)
    r = r / eps
    return r
  elif der_type == 3:
    x_1 = x
    x_2 = x + eps
    r = func(x_2)-func(x_1)
    r = r / eps
    return r